In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

- Цели исследования:
    - проанализировать базу данных приложения для чтения книг
    - найти количество новых книг
    - найти самого популярного автора
    - найти самое большое издательство
    - узнать как пользователи оставляют оценки и обзоры


In [2]:
# Посчитайте, сколько книг вышло после 1 января 2000 года;

query = '''
            SELECT count(*)
            FROM books
            WHERE publication_date > '2000-01-01'
        '''
books_test = pd.io.sql.read_sql(query, con = engine)
print(books_test.loc[0][0],'книг вышло после 1 января 2000 года')

819 книг вышло после 1 января 2000 года


Новых книг в базе данных приложения не так много

In [7]:
# Для каждой книги посчитайте количество обзоров и среднюю оценку;

query = '''
            SELECT 
            books.book_id AS book_id,
            books.title AS title,
            COUNT( DISTINCT reviews.review_id) AS cnt_reviews,
            AVG(ratings.rating) AS avg_rating
            FROM 
            books
            FULL OUTER JOIN ratings ON ratings.book_id = books.book_id
            FULL OUTER JOIN reviews ON books.book_id = reviews.book_id
            GROUP BY
            books.book_id
            
            
        '''
books_test = pd.io.sql.read_sql(query, con = engine)
books_test

,book_id,title,cnt_reviews,avg_rating
0,1,'Salem's Lot,2,3.666667
1,2,1 000 Places to See Before You Die,1,2.500000
2,3,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,4,1491: New Revelations of the Americas Before C...,2,4.500000
4,5,1776,4,4.000000
...,...,...,...,...
995,996,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,997,Xenocide (Ender's Saga #3),3,3.400000
997,998,Year of Wonders,4,3.200000
998,999,You Suck (A Love Story #2),2,4.500000


In [4]:
# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — 
# так вы исключите из анализа брошюры;

query = '''
            SELECT 
            publishers.publisher,
            COUNT(books.book_id) AS cnt_books
            FROM 
            books
            INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
            GROUP BY
            publishers.publisher
            ORDER BY 
            cnt_books DESC
            LIMIT 1
        '''
books_test = pd.io.sql.read_sql(query, con = engine)
print('издательство, которое выпустило наибольшее число книг толще 50 страниц —',books_test.loc[0][0])


издательство, которое выпустило наибольшее число книг толще 50 страниц — Penguin Books


In [5]:
# Определите автора с самой высокой средней оценкой книг — 
# учитывайте только книги с 50 и более оценками;

query = '''
            SELECT
            authors.author,
            AVG(avg_book_ratings.avg_book_rating) as avg_author_rating
            FROM 
            books
            INNER JOIN authors ON authors.author_id = books.author_id
            INNER JOIN (
            SELECT
            book_id,
            AVG(rating) as avg_book_rating
            FROM 
            ratings
            GROUP BY
            book_id
            HAVING count(*) >= 50
            ) AS avg_book_ratings ON avg_book_ratings.book_id = books.book_id
            GROUP BY 
            authors.author
            ORDER BY 
            avg_author_rating DESC
            LIMIT 1
        '''
books_test = pd.io.sql.read_sql(query, con = engine)
print('автор с самой высокой средней оценкой книг —',books_test.loc[0][0])


автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré


In [6]:
# Посчитайте среднее количество обзоров от пользователей, 
# которые поставили больше 50 оценок.

query = '''
            SELECT 
            ROUND(AVG(count)) 
            FROM
            (SELECT
            COUNT(text)
            FROM 
            reviews
            FULL OUTER JOIN (
            SELECT username,
            COUNT(rating_id) as cnt_ratings
            FROM 
            ratings
            GROUP BY
            username
            ) AS cnt_rating ON cnt_rating.username = reviews.username
            WHERE cnt_ratings > 50
            GROUP BY 
            reviews.username) as count;   
            
        '''
books_test = pd.io.sql.read_sql(query, con = engine)

print('среднее количество обзоров от пользователей, которые поставили больше 50 оценок. —',books_test.loc[0][0])


среднее количество обзоров от пользователей, которые поставили больше 50 оценок. — 24.0


##### Вывод
- посчитано количество книг выпушенных после 1 января 2000 года
- создана таблица с расчетом количества обзоров и средней оценки книг
- найдено издательство опубликовавшее большее количество книг более 50 страниц - Penguin Books
- найден автор с самой высокой средней оценкой книг — J.K. Rowling/Mary GrandPré
- посчитано среднее количество обзоров пользователей, оставивших более 50 оценок книгам